# Let's build a Deep NN from scratch

Greatly inspired by deeplearning.ai assignement

**Notation**:
- Superscript $[l]$ denotes a quantity associated with the $l^{th}$ layer. 
    - Example: $a^{[L]}$ is the $L^{th}$ layer activation. $W^{[L]}$ and $b^{[L]}$ are the $L^{th}$ layer parameters.
- Superscript $(i)$ denotes a quantity associated with the $i^{th}$ example. 
    - Example: $x^{(i)}$ is the $i^{th}$ training example.
- Lowerscript $i$ denotes the $i^{th}$ entry of a vector.
    - Example: $a^{[l]}_i$ denotes the $i^{th}$ entry of the $l^{th}$ layer's activations).


## Packages

To build our DNN in python we will use a few useful packages

* **numpy** for mathematical operations
* **matplotlib** to visualize things

In [19]:
import numpy as np
import matplotlib.pyplot as plot
from testCases_v2 import * # for testing purpose

## Initialization

The initialization step is extremely important, there are a lot of different techniques to do it, but the main idea is to use small random numbers to keep w near 0 (to avoid overfitting, and symmetry problems). However, we can initialize b to 0.

We will store the number of units in each layer in an array called `layer_dims`. The shape of the weight matrix is `(layer_dims[l], layer_dims[l-1])` and the shape of the bias vector is `(layer_dims[l], 1)`.

In [40]:

def initialize_parameters_deep(layer_dims):
    """
    @param - layer_dims - python array (list) containing the dimensions of each layer in our network
    @return parameters - python dictionary containing the initialized parameters "W1", "b1", ..., "WL", "bL":
                    Wl - weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl - bias vector of shape (layer_dims[l], 1)
    """
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])* 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        
    return parameters

Example of use:

In [41]:
parameters = initialize_parameters_deep([3,5,4,2])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[-0.01444114 -0.00504466  0.00160037]
 [ 0.00876169  0.00315635 -0.02022201]
 [-0.00306204  0.00827975  0.00230095]
 [ 0.00762011 -0.00222328 -0.00200758]
 [ 0.00186561  0.00410052  0.001983  ]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
W2 = [[ 0.00119009 -0.00670662  0.00377564  0.00121821  0.01129484]
 [ 0.01198918  0.00185156 -0.00375285 -0.0063873   0.00423494]
 [ 0.0007734  -0.00343854  0.00043597 -0.00620001  0.00698032]
 [-0.00447129  0.01224508  0.00403492  0.00593579 -0.01094912]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]]


## Forward propagation

The forward propagation, propagate the data through the network, each neuron has 2 elements:

1. A linear forward module
2. Activation function


Forward propagation gives us $\forall l \in \left \{ 1, 2,\cdots , L \right \}, z^{[l]}\text{ and }a^{[l]}$

### Linear forward module

The linear forward module (vectorized over all the examples) computes the following equation:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{linear}$$

where $A^{[0]} = X$ 

and

$ W = \begin{bmatrix}
\leftarrow & w_0^{[l]^T} &\rightarrow \\
\leftarrow & ... & \rightarrow  \\ 
\leftarrow & w_i^{[l]^T} & \rightarrow  \\
\leftarrow & ... & \rightarrow  \\
\leftarrow & w_{n^{[l]}}^{[l]^T} & \rightarrow  
\end{bmatrix}$

In [33]:
def linear_forward(A,W,b):
    """
    Linear part of a layer's forward propagation
    
    @param  A - activation from the previous layer (size of previous layer, number of examples)
    @param  W - weights matrix (size of current layer)
    @param  b - bias vector, (size of current layer)
    @return Z - input of the activation function (pre-activation parameter)
    @return cache - python tuple containing A, W and b, useful for computing back prop
    """
    Z = np.dot(W,A)+b
    cache = (A, W, b)
    return Z, cache

### Activation function

Each neuron has an activation function,  there are several possibilities : Sigmoid, ReLU, tanh, leaky ReLU ... Here, we are going to use ReLU and sigmoid as activation functions.

In [43]:
def sigmoid(Z):
    """
    @param  Z     - numpy array of any shape
    @return A     - output of sigmoid(Z), same shape as Z
    @return cache - return Z, useful during backpropagation
    """
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    """
    @param Z - output of the linear layer, any shape
    @return A - ReLU(Z), same shape as Z
    @return cache - return Z, useful during backprop
    """
    A = np.maximum(0,Z)
    cache = Z
    assert(A.shape == Z.shape)
    return A, cache


For more convenience, we are going to group all these functions (linear and activation) into one function.

In [47]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Forward propagation
    
    @param A_prev     - Activations from previous layer (size of prev layer, number of examples)
    @param W          - Weights matrix (size of current layer, size of previous layer)
    @param b          - Bias vector (size of current layer, 1)
    @param activation - String representing the activation function to use
    @return A         - The result, post-activation value
    @return cache     - linear_cache and activation_cache
    """
    Z, linear_cache = linear_forward(A_prev, W, b)
    if activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    return A, cache

In [48]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


### Our model

For even more convenience when implementing an L-layer NN, we will need a function that replicates the previous one `linear_activation_forward` L times. Here, we define the model of our DNN.

In this example, we are going to use L-1 layers composed of ReLUnits, and the last layer will use one sigmoid unit.


In [49]:
def L_model_forward(X, parameters):
    """
    Forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    @param X          - input data (input size, number examples)
    @param parameters - output of initalize_parameters_deep()
    @return AL        - last post-activation (L-1 firsts)
    @return caches    - caches containing every cache of linear_relu_forward() and linear_sigmoid_forward (last one)
    
    """
    caches = []
    A = X
    L = len(parameters)//2 # number of layers in the nn
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches"
    for l in range(1,L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], "relu")
        caches.append(cache)
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches"
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")
    caches.append(cache)
    
    return AL, caches

In [50]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.17007265 0.2524272 ]]
Length of caches list = 2


That was easy, we now have a full forward propagation that takes the input X and outputs a row vector $A^{[L]}$ containing our predictions. It also records all intermediate values in "caches". Using $A^{[L]}$, we can compute the cost of our predictions.

## Cost function

It's time to measure how well our NN performed. We will use the cross-entropy cost J to do that :

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right))$$

In [51]:
def compute_cost(AL, Y):
    """
    Compute the cost function
    @param AL    - probability vector corresponding to our predictions (1, number of examples)
    @param Y     - True label vector (1, number of examples)
    @return cost - cross-entropy cost
    """
    m = Y.shape[1]
    cost = -1*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))/m
    cost = np.squeeze(cost) # to make sure cost's shape is what we expect
    return cost

Remember: `*` is an element wise multiplication (we also can use `np.multiply()`, while `np.dot()` is the dot product (matrix product).

## Backpropagation

The idea of backpropagation is to compute the derivates of the cost function with respect to w, to be able to update the values of
all w. In order to do that, we use an interesting property called "Chain rule".

![The best chain rule diagram ever made](chainrule.png)

We can compute $\frac{\partial J}{\partial w}$ by chaining (multiplying) the local partial derivatives. Example, imagine J is compute just after $a^{[l]}$:


$$\frac{\partial J}{\partial w^{[l-1]}} = \frac{\partial J}{\partial a^{[l]}} \cdot \frac{\partial a^{[l]}}{\partial z} \cdot \frac{\partial z^{[l]}}{\partial a^{[l-1]}} \cdot \frac{\partial a^{[l-1]}}{\partial z^{[l-1]}} \cdot \frac{\partial z^{[l-1]}}{\partial w^{[l-1]}} $$

To simplify this expression we introduce the following notation:

$dA^{[l]} = \frac{\partial J}{\partial a^{[L]}}\cdot\frac{\partial a^{[L]}}{\partial z^{[L]}} \cdot\cdot\cdot \frac{\partial z^{[l+1]}}{\partial a^{[l]}}$

$dZ^{[l]} = dA \cdot \frac{\partial a^{[l]}}{\partial z^{[l]}}$

$dw^{[l]} = dZ^{[l]} \cdot \frac{\partial z^{[l]}}{\partial w^{[l]}}$

$db^{[l]} = dZ^{[l]} \cdot \frac{\partial z^{[l]}}{\partial b^{[l]}}$

If we do a little bit of calculus, we get the following expressions:

$dw^{[l]} = dZ^{[l]} \cdot a^{[l-1]}$

$db^{[l]} = dZ^{[l]}$

$dA^{[l-1]} = w^{[l]^T}\cdot dZ^{[l]}$

Our job now is to compute these values.


### Linear backward

For layer l, the linear part is $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$

Suppose we have already calculated the derivative $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. We can easily get $(dW^{[l]}, db^{[l]} dA^{[l-1]})$, using the vectorized version :

$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$$

In [53]:
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    @param dZ    - Gradient of the cost with respect to the linear output (of current layer l)
    @param cache - tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    
    @return dA_prev - Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    @return dW      - Gradient of the cost with respect to W (current layer l), same shape as W
    @return db      - Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dA_prev = np.dot(W.T,dZ)
    dW = np.dot(dZ, A_prev.T)/m
    db = np.sum(dZ,axis=1, keepdims=True)/m

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [54]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


### Linear-Activation backward

To help implement linear_activation_backward, we provide two backward functions:

* `sigmoid_backward`
* `relu_backward`

to compute $dZ$.

In [57]:
def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    @param  dA    - post-activation gradient, of any shape
    @param  cache - Z stored for computing backward propagation efficiently
    @return dZ    - Gradient of the cost with respect to Z
    """
    
    Z = cache    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    return dZ

In [58]:
def relu_backward(dA, cache):
    """
    Implement backprop for a single ReLU unit
    
    @param dA    - post-activation gradient, of any shape (previous layer)
    @param cache - Z computed during forward propagation
    @return dZ   - Gradient of the cost with respect to Z
    """
    Z = cache
    dZ = np.array(dA, copy=True)# just converting dz to a correct object
    # if Z <= 0 then dZ = 0 else dZ is just the same (=dA)
    dZ[Z <= 0] = 0
    return dZ

We will implement the backward propagation for one layer, from $dA^{[l]}$ it will output $dA^{[l-1]}, dW^{[l]}$ and $db^{[l]}$

In [60]:

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    @param dA         - post-activation gradient for current layer l 
    @param cache      - (linear_cache, activation_cache) used for computing backward propagation efficiently
    @param activation - the activation to be used in this layer, String: "sigmoid" or "relu"
    
    @return dA_prev   - Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    @return dW        - Gradient of the cost with respect to W (current layer l), same shape as W
    @return db        - Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [61]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


### L-Model backward prop

Now we are done with one layer, we can repeat these operations to implement our model. Recall that when we implemented the `L_model_forward` function, at each iteration, we stored a cache which contains (X,W,b,z). In the back prop module, we will use this cache to compute the gradients.

The first thing we need to do before backpropagating is to compute the first gradient $ dA^{[L]}= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$ = dAL where $A^{[L]} = \sigma(Z^{[L]})$ :

$$dA^{[L]} = \frac{\partial \mathcal{L}}{\partial A^{[L]}} = \frac{Y}{ A^{[L]}} - \frac{1-Y}{1-A^{[L]}}$$

We can then use dAL to keep going backward.

In [84]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = -(np.divide(Y, AL) - np.divide(1-Y, 1-AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, caches[-1], activation="sigmoid")
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        grads["dA" + str(l + 1)], grads["dW" + str(l + 1)], grads["db" + str(l + 1)] = linear_activation_backward(grads["dA" + str(l + 2)], caches[l], activation="relu")

    return grads


In [85]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


## Update parameters

The hardest is done, we just need to update our parameters, and we are DONE with the building blocks, then we will be able to put everything together, train our model and make predictions (let's gooooooooo).

We update our params like dat :

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$$

where $\alpha$ is the learning rate. After computing the updated parameters, store them in the parameters dictionary. 

In [108]:

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    @param parameters - python dictionary containing the parameters 
    @param grads      - python dictionary containing the gradients, output of L_model_backward
    
    Returns:
    @return parameters  - python dictionary containing the updated parameters 
                          parameters["W" + str(l)] = ... 
                          parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate*grads["dW"+str(l+1)]  
        parameters["b" + str(l+1)] -= learning_rate*grads["db"+str(l+1)]
        
    return parameters

In [109]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
